In [9]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

In [10]:
num_random_seeds = 5
num_bootstrap_models = 100
MODEL_CLASS = "lr" #lr,nn,rf
TARGET_VARIABLE = "qualified_gagne_3"

In [11]:
data_source = "data/obermeyer/obermeyer_data_cleaned.csv"
prediction_output = "predictions/obermeyer/test_bootstrap_"+MODEL_CLASS+"_"+TARGET_VARIABLE+".csv"
features = ['dem_female', 'dem_age_band_18-24_tm1', 'dem_age_band_25-34_tm1', 'dem_age_band_35-44_tm1', 'dem_age_band_45-54_tm1',
            'dem_age_band_55-64_tm1', 'dem_age_band_65-74_tm1', 'dem_age_band_75+_tm1', 'hypertension_elixhauser_tm1', 'cost_dialysis_tm1',
            'cost_emergency_tm1', 'cost_home_health_tm1', 'cost_ip_medical_tm1', 'cost_ip_surgical_tm1', 'cost_laboratory_tm1',
            'cost_op_primary_care_tm1', 'cost_op_specialists_tm1', 'cost_op_surgery_tm1', 'cost_other_tm1', 'cost_pharmacy_tm1',
            'cost_physical_therapy_tm1', 'cost_radiology_tm1', 'gagne_sum_tm1']
other_variables = ['person_id', 'gagne_sum_t', 'cost_t']

In [12]:
df = pd.read_csv(data_source)
X = df[features+other_variables]
y = df[TARGET_VARIABLE]

In [13]:
def get_bootstrap_model(X_train, y_train, X_test, random_seed, bootstrap_size=0.5):
    np.random.seed(random_seed)
    idx = np.random.permutation(len(X_train))[:int(len(X_train)*bootstrap_size)]
    if MODEL_CLASS == "lr":
        model = LogisticRegression()
        #model = LinearRegression()
    elif MODEL_CLASS == "rf":
        model = RandomForestClassifier(random_state=random_seed)
        #model = RandomForestRegressor(random_state=random_seed)
    elif MODEL_CLASS == "nn":
        model = MLPClassifier(random_state=random_seed)
        #model = MLPRegressor(random_state=random_seed)
    model.fit(X_train[idx], y_train[idx])
    return model.predict_proba(X_test)[:, 1]

In [14]:
output = []
for random_seed in range(num_random_seeds):
    print("random seed", random_seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=random_seed)

    cost = X_test["cost_t"].to_numpy()
    gagne = X_test["gagne_sum_t"].to_numpy()
    person_id = X_test['person_id'].to_numpy()
    
    X_train = X_train.drop(columns=other_variables).to_numpy()
    y_train = y_train.to_numpy()
    X_test = X_test.drop(columns=other_variables).to_numpy()
    y_test = y_test.to_numpy()

    predictions = []
    columns = []
    for i in tqdm(range(num_bootstrap_models)):
        columns.append(f'm_{i+1}')
        scores = get_bootstrap_model(X_train, y_train, X_test, random_seed=i, bootstrap_size=0.5)
        predictions.append(scores)
    
    predictions_df = pd.DataFrame(predictions).transpose()
    predictions_df.columns=columns
    predictions_df["y"] = y_test
    predictions_df["person_id"] = person_id 
    predictions_df["seed"] = random_seed
    
    predictions_df['cost_t'] = cost
    predictions_df['gagne_sum_t'] = gagne

    output.append(predictions_df)
output = pd.concat(output)

random seed 0


100%|█████████████████████████████████████████| 100/100 [00:01<00:00, 63.03it/s]


random seed 1


100%|█████████████████████████████████████████| 100/100 [00:01<00:00, 65.92it/s]


random seed 2


100%|█████████████████████████████████████████| 100/100 [00:01<00:00, 64.35it/s]


random seed 3


100%|█████████████████████████████████████████| 100/100 [00:01<00:00, 64.44it/s]


random seed 4


100%|█████████████████████████████████████████| 100/100 [00:01<00:00, 64.62it/s]


In [15]:
print(output.info())

<class 'pandas.core.frame.DataFrame'>
Index: 80495 entries, 0 to 16098
Columns: 105 entries, m_1 to gagne_sum_t
dtypes: float64(101), int64(4)
memory usage: 65.1 MB
None


In [16]:
output.to_csv(prediction_output, index=False)